# **WIP Collaborative Filtering**

#Step 1: Selection / Read case / Install Done
#Step 2: Data review (type) / EDA Done
#Step 3: JOIN link md and ratings Done
#Step 4: EDA Done
#Step 5: run Colab filtering Done
#Step 6: Clean / Iterate Done
#Step 7: Predict Done

#Step 8: Check if item is indeed in top list of user 36. And not in list
#Step 9: Run augmented code to see recommended movies
#Step 10: Check if recommended movies make sense
#Step TBD: Check what went wrong with the list of movies joined

#Step 11: Github / Derive Code for dashboard
#Step 12: Run dashboard
#Step 13: Iterate/clean
#Step 14: Github

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [ ]:
#DOWNLOAD data to dataframe
reader = Reader()
#ratings = pd.read_csv('../input/tmdb-ratings_small/tmdb_5000_movies.csv')
rd = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
md= pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
ld= pd.read_csv('/kaggle/input/the-movies-dataset/links.csv')
#ratings.head()
rd.head(3)

In [ ]:
#metadata head
md.head(3)


In [ ]:
#EXPLORE link data head
ld.head(3)


In [ ]:
#CHECK NULL VALUES
#null_values = rd.isnull()
#print(null_values)
rd.info()
ld.info()
md.info()

In [ ]:
# MERGE rd with ld so as to append a linking variable tmdbId that can connect with md (md has movie titles)
rd_ld = pd.merge(rd, ld, on='movieId')
rd_ld.head(5)
rd_ld.info()



In [ ]:
#RENAME new rd_ld DB's ID so as to mirror md's ID before merging
rd_ld = rd_ld.rename(columns={'tmdbId': 'id'})
rd_ld.head(5)  # Output:   new_column

In [ ]:
# CONVERT md ID column to numeric before merging
md['id'] = pd.to_numeric(md['id'], errors='coerce')
print(md['id'].dtype)  # Output: float64
md.info()

In [ ]:
# MERGED DATASET - eventually didn't help me much either LOL
# MERGE rd ld md so that userID movie name and ratings can all be connected in same file
rd_ld_md = pd.merge(rd_ld, md, on='id')
movie_ratings_joined = rd_ld_md[['userId', 'movieId','id', 'title', 'budget','genres', 'popularity','revenue', 'production_countries', 'vote_average', 'rating']]
#print(movie_ratings_joined)
movie_ratings_joined.tail(5)
#movie_ratings_joined.info()

In [ ]:
movie_ratings_joined.shape
#movie_ratings_joined.info()


In [ ]:
# EDA (didn't help me much)
plt.figure(figsize=(8, 6))
plt.scatter(movie_ratings_joined['revenue'], movie_ratings_joined['vote_average'])

# Add title and labels
plt.title('Scatterplot of revenue vs vote_average')
plt.xlabel('popularity')
plt.ylabel('vote_average')

# Display the plot
plt.show()


In [ ]:
# HISTOGRAM (didn't help me much)
plt.hist(movie_ratings_joined['rating'], bins=11, alpha=0.7, color='skyblue', edgecolor='black')

# Add title and labels
plt.title('Histogram of ratings')
plt.xlabel('ratings')
plt.ylabel('Frequency')

# Display the plot
plt.show()

In [ ]:
#CONVERT some fields to numeric from the merged dataset
movie_ratings_joined['popularity'] = pd.to_numeric(movie_ratings_joined['popularity'], errors='coerce')
print(movie_ratings_joined['popularity'].dtype)  # Output: float64
movie_ratings_joined.info()

movie_ratings_joined['vote_average'] = pd.to_numeric(movie_ratings_joined['vote_average'], errors='coerce')
print(movie_ratings_joined['vote_average'].dtype)  # Output: float64
movie_ratings_joined.info()

In [ ]:
# DATA REVIEW CHECKS

#Run barchartwith of production origin 

#index = md[md['id'] == '1029'].index[0]
#print(f"Value 'id' found at index {index}")
#index = md.index('139405')
#Print(f"Value '139405' found at index {index}")
#row = rd.loc[rd['movieId'] == 31]
#print(row)
#id_map = pd.read_csv('../input/links_small.csv')[['movieId', 'tmdbId']]
#id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
#id_map.columns = ['movieId', 'id']
#id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

#check of TMBID can be found in md
row = md.loc[md['id'] == 9909]
row.head()

#check how many null is there
missing_values = movie_ratings_joined.isnull().sum()
#missing_values = movie_ratings_joined.isnull()
print(missing_values)
movie_ratings_joined.info()

In [ ]:
# EDA (didn't help me much)

# Create figure and axis
##fig, ax = plt.subplots(figsize=(10, 6))

# Plot bar chart
#ax.bar(movie_ratings_joined['production_countries'], movie_ratings_joined['vote_average'].mean())

#average_values = movie_ratings_joined.groupby('production_countries')['vote_average'].mean()
##ax.bar(movie_ratings_joined['production_countries'], movie_ratings_joined.groupby('production_countries')['vote_average'].mean())
#SHOULD TRY SQL HERE

# Set title and labels
##ax.set_title('Vote avg by country')
##ax.set_xlabel('Country')
##ax.set_ylabel('Vote AVG')

# Rotate x-axis labels for better readability
##plt.xticks(rotation=45)

# Show plot
##plt.show()

In [ ]:
# RECOMMENDER BUILDING leading up to predicted ratings
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Reader

#df = pd.DataFrame(ratings_dict)
#df = pd.DataFrame(movie_ratings_joined)
#Define the reader
#reader = Reader(rating_scale=(1, 5))
reader = Reader()

# Build the dataset
#data = Dataset.load_from_df(movie_ratings_joined[["userId", "title", "rating"]], reader)
#data = Dataset.load_from_df(movie_ratings_joined[['userId', 'title', 'rating']], reader)
#data = Dataset.load_from_df(df[['userId', 'title', 'rating']], reader)
data = Dataset.load_from_df(rd[['userId', 'movieId', 'rating']], reader)


# Split the dataset into training and testing sets
Trainset, Testset = train_test_split(data, test_size=.25)

# Create a KNNWithMeans recommender
algo = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False})

# Train the recommender
algo.fit(Trainset)

# Make predictions on the test set
predictions = algo.test(Testset)
#predictions = algo.test(Trainset)
# Evaluate the recommender
accuracy.rmse(predictions, verbose=True)  # RMSE (Root Mean Square Error)
accuracy.mae(predictions, verbose=True)   # MAE (Mean Absolute Error)

# Make a prediction for a specific user-item pair
##prediction = algo.predict('12', '2', r_ui=4, verbose=False)
##print(prediction.est)  # Output: 4.25


In [ ]:
# PREDICTION
prediction = algo.predict(36, 32, r_ui=0, verbose=False)
print(prediction.est)  # Output: 3.54066

In [ ]:
# CHECK ON TOP RECOMMENDATIONS OF USER 547
#rd.loc[rd['userId'] == 547]
rd547=rd.loc[rd['userId'] == 547].sort_values(by='rating', ascending=False)

#Use the value_counts() function to count the frequency of each userId
##user_id_counts = rd['userId'].value_counts()

#Print the top 5 userIds with the highest frequency
##print(user_id_counts.nlargest(5))


In [ ]:
# RECOMMENDER BUILDING + TOP 10 RECOMMENDATIONS
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

# Load the dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rd[['userId', 'movieId', 'rating']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=.25)

# Create a KNNWithMeans algorithm object
algo = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False})

# Train the algorithm on the training set
algo.fit(trainset)

# Make predictions on the test set
predictions = algo.test(testset)

# Function to get Top-N recommendations for a user
def get_top_n_recommendations(user_id, n=10):
    # Get the user's ratings
    user_ratings = [r for r in trainset.all_ratings() if r[0] == user_id]

    # Get the user's rated items
    rated_items = [r[1] for r in user_ratings]

    # Get the predicted ratings for the user
    predicted_ratings = []
    for item in trainset.all_items():
        if item not in rated_items:
            predicted_rating = algo.predict(user_id, item, r_ui=0, verbose=False).est
            predicted_ratings.append((item, predicted_rating))

    # Sort the predicted ratings in descending order
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)

    # Return the Top-N recommended items
    return predicted_ratings[:n]

# Example usage
user_id = 32  # Replace with the desired user ID
n = 10  # Replace with the desired number of recommendations

top_n_recommendations = get_top_n_recommendations(user_id, n)
print("Top-{} recommendations for user {}".format(n, user_id))
for item, rating in top_n_recommendations:
    print("Item: {}, Predicted Rating: {}".format(item, rating))

In [ ]:
# SEE MOVIE TITLES of top recommendations
top_recommendations = pd.DataFrame(top_n_recommendations, columns=['movieId', 'Rating'])
top_recommendations.head(20)

#mergedRD = pd.merge(movie_ratings_joined, top_recommendations, on='movieId')
#mergedRD.head(20)

mergedRD = pd.merge(top_recommendations[['movieId']], movie_ratings_joined[['movieId','title']], on='movieId', how='left')
df_unique = mergedRD.drop_duplicates()
df_unique

In [ ]:
#SURPLUS CODE
#rdconcise = rd[['userId', 'movieId', 'rating']]
#rdconcise.loc[rdconcise['movieId'] == 30]
#print(row)
#rd.loc[rd['movieId'] == 559]

#To augment the Python code to include movie titles in the Top-N recommended items, you'll need to make the following modifications:

#Step 1: Merge Movie Titles with the Dataset
#Assuming you have a DataFrame movies_df containing the movie IDs and titles, merge this DataFrame with your original ratings DataFrame your_dataframe before loading it into the Surprise Dataset.
#merged_df = pd.merge(your_dataframe, movies_df, on='movieId')
##movie_ratings_joined


#Step 2: Modify the get_top_n_recommendations Function
#Update the get_top_n_recommendations function to include the movie titles in the predicted ratings list.


#def get_top_n_recommendations(user_id, n=10):
    # Get the user's ratings
#    user_ratings = [r for r in trainset.all_ratings() if r[0] == user_id]

    # Get the user's rated items
#    rated_items = [r[1] for r in user_ratings]

    # Get the predicted ratings for the user
#    predicted_ratings = []
#    for item in trainset.all_items():
#        if item not in rated_items:
#            predicted_rating = algo.predict(user_id, item, r_ui=0, verbose=False).est
            # Get the movie title from the merged DataFrame
#            movie_title = movie_ratings_joined.loc[movie_ratings_joined['movieId'] == item, 'title'].iloc[0]
#            predicted_ratings.append((item, movie_title, predicted_rating))

    # Sort the predicted ratings in descending order
#    predicted_ratings.sort(key=lambda x: x[2], reverse=True)

    # Return the Top-N recommended items
#    return predicted_ratings[:n]



#Step 3: Update the Example Usage
#Modify the example usage to print the movie titles along with the predicted ratings.
#user_id = 11  # Replace with the desired user ID
#n = 10  # Replace with the desired number of recommendations

#top_n_recommendations = get_top_n_recommendations(user_id, n)
#print("Top-{} recommendations for user {}".format(n, user_id))
#for item, title, rating in top_n_recommendations:
#    print("Item: {}, Title: {}, Predicted Rating: {}".format(item, title, rating))